In [36]:
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import torch
import evaluate  # pip install evaluate
import seqeval   # pip install seqeval
from datasets import load_dataset

In [2]:
model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', num_labels=7)

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [4]:
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [5]:
# 模型测试
message= "命名实体识别"
label = torch.tensor([0,1,0,2,5,4])

model_input = tokenizer([message], return_tensors='pt')
result = model(**model_input)

print(result.loss)
print(result.logits)

None
tensor([[[ 0.3466,  0.0680,  0.1839,  0.3548,  0.1633,  0.2896,  0.4516],
         [ 0.1477, -0.2928,  0.4195, -0.0412, -0.1842,  0.3515,  0.5917],
         [-0.0289,  0.1072,  0.5445,  0.1000,  0.0958,  0.8696,  0.1344],
         [-0.1187, -0.1498,  0.2413, -0.3251, -0.0966,  0.6368, -0.3206],
         [-0.1577, -0.0087,  0.7181, -0.0674,  0.0150,  0.5987, -0.4756],
         [-0.2419, -0.3130, -0.3713, -0.5937,  0.0140,  0.5136,  0.1689],
         [-0.0920, -0.1692,  0.4886, -0.0962,  0.2686,  0.3336, -0.0947],
         [ 0.5450,  0.1952,  0.2713, -0.2293,  0.3201,  0.4604,  0.3622]]],
       grad_fn=<ViewBackward0>)


In [6]:
# 加载hf中dataset
ds = load_dataset('nlhappy/CLUE-NER')
ds

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

(…)-00000-of-00001-a33d0e4276aef9b4.parquet:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

(…)-00000-of-00001-07f476b71c5edde6.parquet:   0%|          | 0.00/178k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10748 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1343 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'ents'],
        num_rows: 10748
    })
    validation: Dataset({
        features: ['text', 'ents'],
        num_rows: 1343
    })
})

## 实体映射数据集词典准备

In [7]:
# entity_index
entites = ['O'] + list({'movie', 'name', 'game', 'address', 'position', \
           'company', 'scene', 'book', 'organization', 'government'})
tags = ['O']
for entity in entites[1:]:
    tags.append('B-' + entity.upper())
    tags.append('I-' + entity.upper())

entity_index = {entity:i for i, entity in enumerate(entites)}


In [8]:
entity_index

{'O': 0,
 'government': 1,
 'position': 2,
 'company': 3,
 'movie': 4,
 'address': 5,
 'book': 6,
 'scene': 7,
 'organization': 8,
 'game': 9,
 'name': 10}

In [9]:
tags

['O',
 'B-GOVERNMENT',
 'I-GOVERNMENT',
 'B-POSITION',
 'I-POSITION',
 'B-COMPANY',
 'I-COMPANY',
 'B-MOVIE',
 'I-MOVIE',
 'B-ADDRESS',
 'I-ADDRESS',
 'B-BOOK',
 'I-BOOK',
 'B-SCENE',
 'I-SCENE',
 'B-ORGANIZATION',
 'I-ORGANIZATION',
 'B-GAME',
 'I-GAME',
 'B-NAME',
 'I-NAME']

In [10]:
def entity_tags_proc(item):
    # item即是dataset中记录
    text_len = len(item['text'])  # 根据文本长度生成tags列表
    tags = [0] * text_len    # 初始值为‘O’
    # 遍历实体列表，所有实体类别标记填入tags
    entites = item['ents']
    for ent in entites:
        indices = ent['indices']  # 实体索引
        label = ent['label']   # 实体名
        tags[indices[0]] = entity_index[label] * 2 - 1
        for idx in indices[1:]:
            tags[idx] = entity_index[label] * 2
    return {'ent_tag': tags}

# 使用自定义回调函数处理数据集记录
ds1 = ds.map(entity_tags_proc)

Map:   0%|          | 0/10748 [00:00<?, ? examples/s]

Map:   0%|          | 0/1343 [00:00<?, ? examples/s]

In [11]:
# 训练集
for row in ds1['train']:
    print(row['text'])
    print(row['ent_tag'])
    
    break

浙商银行企业信贷部叶老桂博士则从另一个角度对五道门槛进行了解读。叶老桂认为，对目前国内商业银行而言，
[5, 6, 6, 6, 0, 0, 0, 0, 0, 19, 20, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
ds1

DatasetDict({
    train: Dataset({
        features: ['text', 'ents', 'ent_tag'],
        num_rows: 10748
    })
    validation: Dataset({
        features: ['text', 'ents', 'ent_tag'],
        num_rows: 1343
    })
})

中文bert分词在日期时间和英文转换token过程中，出现合并。影响ner标注准确性。

In [13]:
token_index = tokenizer.encode('2000年2月add', add_special_tokens=False)
print(token_index)
tokens = tokenizer.decode(token_index)
print(tokens)

[8202, 2399, 123, 3299, 10253]
2000 年 2 月 add


In [14]:
input_data = tokenizer(['2000年2月add'], add_special_tokens=False, truncation=True)
print(input_data)

input_data.word_ids(0) # 返回批次中指定token对应原始文本的索引映射

{'input_ids': [[8202, 2399, 123, 3299, 10253]], 'token_type_ids': [[0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1]]}


[0, 1, 2, 3, 4]

In [15]:
# 原始文本转换模型需要token_idx,生成和token_idx对齐label
def data_input_proc(item):
    # 输入文本转换模型输入token索引
    input_data = tokenizer(item['text'], truncation=True, add_special_tokens=False, max_length=512)
    adjust_labels = []  # 所有修正后label索引列表
    # 上一步骤生成ner_tag中索引和token对齐
    for k in range(len(input_data['input_ids'])):
        # 每条记录token对应word_ids
        word_ids = input_data.word_ids(k)
        # 批次ner_tag长度和token长度对齐
        tags = item['ent_tag'][k]
        
        adjusted_label_ids = []
        i, prev_wid = -1,-1
        for wid in word_ids:
            if (wid != prev_wid):   #  word_ids [1,1,1,2,3,4,5] -> [0,1,2,3,4,5,6]
                i += 1 # token对应检索位置+1
                prev_wid = wid
            adjusted_label_ids.append(tags[i])
        adjust_labels.append(adjusted_label_ids)                
    # 修正后label添加到input_data
    input_data['labels'] = adjust_labels
    return input_data
    

ds2 = ds1.map(data_input_proc, batched=True)  # batched 每次传入自定义方法样本数量多个

Map:   0%|          | 0/10748 [00:00<?, ? examples/s]

Map:   0%|          | 0/1343 [00:00<?, ? examples/s]

In [16]:
# 记录转换为pytorch
ds2.set_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
# ds_new = ds2.with_format('torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [17]:
# for item in ds2['train']:
#     print(item)
#     break

## 模型训练
### TrainingArguments


In [18]:
args = TrainingArguments(
    output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
    num_train_epochs = 3,    # 训练 epoch
    save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
    per_device_train_batch_size=32,  # 训练批次
    per_device_eval_batch_size=32,
    report_to='tensorboard',  # 训练输出记录
    eval_strategy="epoch",
)

### Model

In [19]:
id2lbl = {i:tag for i, tag in enumerate(tags)}
lbl2id = {tag:i for i, tag in enumerate(tags)}

model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', 
                                                        num_labels=21,
                                                        id2label=id2lbl,
                                                        label2id=lbl2id)
model

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

## Trainer

In [48]:
# metric 方法
def compute_metric(result):
    # result 是一个tuple (predicts, labels)
    
    # 获取评估对象
    seqeval = evaluate.load('seqeval')
    predicts,labels = result
    predicts = np.argmax(prdicts, axis=2)
    
    # 准备评估数据
    predicts = [[tags[p] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    results = seqeval.compute(predictions=predicts, references=labels)

    return results


In [49]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)

trainer = Trainer(
    model,
    args,
    train_dataset=ds2['train'],
    eval_dataset=ds2['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metric
)

模型训练

In [50]:
trainer.train()

Step,Training Loss
500,0.175500


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=504, training_loss=0.17525818352661435, metrics={'train_runtime': 204.5277, 'train_samples_per_second': 157.651, 'train_steps_per_second': 2.464, 'total_flos': 820469833815600.0, 'train_loss': 0.17525818352661435, 'epoch': 3.0})

模型推理

In [26]:
result = trainer.predict(ds2['validation'])

In [39]:
print(ds1['validation'][10]['text'])
print(ds2['validation'][10]['labels'])
print(result.label_ids[10])

根据北京市消防局的说法，此次火灾主要原因是责任单位违规燃放礼花弹，燃放期间民警多次劝阻未果。
tensor([0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0])
[   0    0    1    2    2    2    2    2    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    3    4    0    0    0
    0    0    0    0 -100 -100 -100 -100]


In [46]:
 [tags[p] for p,l in zip(result.label_ids[10],ds2['validation'][10]['labels'])]

['O',
 'O',
 'B-GOVERNMENT',
 'I-GOVERNMENT',
 'I-GOVERNMENT',
 'I-GOVERNMENT',
 'I-GOVERNMENT',
 'I-GOVERNMENT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-POSITION',
 'I-POSITION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [47]:
[tags[l] for p,l in zip(result.label_ids[10],ds2['validation'][10]['labels'])]

['O',
 'O',
 'B-GOVERNMENT',
 'I-GOVERNMENT',
 'I-GOVERNMENT',
 'I-GOVERNMENT',
 'I-GOVERNMENT',
 'I-GOVERNMENT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-POSITION',
 'I-POSITION',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']